<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

23 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-02-28 21:00:51--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?U1ZlrykJlsPt5kOOjBCMU3Zg3B9DLqCwNEQPas9EJrBPCLFPYZXFzq-Mw9roCefo0Fk8VXg-w5oQAvUv2lSWla3XRY2KYL71yrHmLDjiMVXEACoNxze4K0tEV3Uv5u7KUBo5mB3xs7LZgupislp_7RnOCFcH8CVXn6mC8PONAb5ZcockgLSeGAPrA5X7fpQ4qhOFGC7smWEWfLHTsZQpyqwF5Q [following]
--2019-02-28 21:00:51--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?U1ZlrykJlsPt5kOOjBCMU3

In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-b235klhe
  Stored in directory: /tmp/pip-ephem-wheel-cache-dgfsbtd9/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

CODE (AT LEAST NOW)

In [0]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [7]:
%%cu
#include <stdio.h>

int main() {
	int deviceCount;
  cudaDeviceProp devProp;
    
  cudaGetDeviceCount(&deviceCount);
  printf("Found %d devices\n", deviceCount);
	return 0;
}


'Found 1 devices\n'

In [87]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <fstream>
#include <math.h>
#include <cmath>

using namespace std;

#define ITERATIONS 1000
#define N 64

#define e 2.71
#define Dx 1
#define h ((float) Dx / N * 1.0)
#define h2 powf(h, 2.0)

#define max(x, y) (((x) > (y)) ? (x) : (y))
#define min(x, y) (((x) < (y)) ? (x) : (y))

#define r1 64
#define r2 64

#define t1 64
#define t2 64

__device__
float f(int i, int j) {
	return 2 * powf(e, i * h + j * h);
}

float u_t(float x) {
	return powf(e, x);
}

float u_l(float x) {
	return powf(e, x);
}

float u_r(float x) {
	return powf(e, x + 1);
}

float u_d(float x) {
	return powf(e, x + 1);
}


__device__
float u(float* matrix, int i, int j) {
	return matrix[N * i + j];
}

__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * r1 + threadIdx.x * t1;
	int jBegin = jGl * r2 + threadIdx.y * t2;

  //printf("iGl : %d, %d, %d, %d, %d\n", iGl, jGl, threadIdx.x, threadIdx.y, isBlack);
  //printf("iGl : %d, %d \n", iBegin, jBegin);

	for (size_t i = max(iBegin + 1, 1); i < min(iBegin + t1, N - 1); i++) {
		int offset = i % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
		for (size_t j = max(jBegin + offset + 1, 1); j < min(jBegin + t2, N - 1); j+=2) {
			matrix[N * i + j] = 0.25 * (
          u(matrix, i + 1, j)
				+ u(matrix, i - 1, j)
				+ u(matrix, i, j + 1)
				+ u(matrix, i, j - 1) - h2 * f(i, j));
		}
	}
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createMatrix(int row, int col) {
	float* matrix = new float[row * col];
	for(size_t i = 0; i < row; i++) {
		for(size_t j = 0; j < col; j++) {
			matrix[i * row + j] = 0;
		}
	}
    
  for(size_t i = 1; i < row - 1; i++) {
    matrix[0 * row + i] = u_t(i * h);
    matrix[(row - 1) * row + i] = u_d(i * h);
	}

  for(size_t j = 1; j < col - 1; j++) {
    matrix[j * col + 0] = u_l(j * h);
    matrix[j * col + row - 1] = u_r(j * h);
	}
    
	return matrix;
}

float calculateDifference(float* matrix1) {
	float tolerance = 0.0f;
	for (size_t i = 0; i < N * N; i++) {
    float fCurrent = func(i / N, i % N);
		tolerance += (matrix1[i] - fCurrent) * (matrix1[i] - fCurrent);
	}
  
	return tolerance;
}

int main() {
  ofstream myfile;
  myfile.open("example.txt");
	float* matrixHost = createMatrix(N, N);
  

  for (size_t i = 0; i < N; i++) {
		for (size_t j = 0; j < N; j++) {
      myfile << matrixHost[i * N + j] << " ";
		}
    myfile << "\n";
	}

  

	float* matrixDevice;
	
	int matrixSize = sizeof(float) * N * N;

	cudaMalloc((void**)&matrixDevice, matrixSize);
	cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice);

	long startTime = clock();
	dim3 blocks(N / r1, N / r2);
	dim3 threads(r1 / t1, r2 / t2);

	for (size_t i = 0; i < ITERATIONS; i++) {
		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

		calculateBlack <<< blocks, threads >>>(matrixDevice);
		cudaDeviceSynchronize();
		calculateRed <<< blocks, threads >>>(matrixDevice);

		cudaPeekAtLastError();
		cudaDeviceSynchronize();

		cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

    cout << calculateDifference(matrixHost) << "\n";
    for (size_t i = 0; i < N; i++) {
		  for (size_t j = 0; j < N; j++) {
        myfile << matrixHost[i * N + j] << " ";
		  }
      myfile << "\n";
	 }
     
	}
	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}
  myfile.close();
	return 0;
}


'35281.8\n33860.8\n32821.6\n31970.7\n31238\n30588.1\n30000.9\n29462.7\n28964.6\n28499.7\n28063.2\n27651.2\n27260.4\n26888.6\n26533.6\n26193.8\n25867.7\n25554.1\n25251.9\n24960.2\n24678.2\n24405.2\n24140.6\n23883.7\n23634.2\n23391.6\n23155.4\n22925.2\n22700.9\n22482\n22268.2\n22059.3\n21855.1\n21655.3\n21459.8\n21268.3\n21080.7\n20896.8\n20716.4\n20539.5\n20365.8\n20195.4\n20027.9\n19863.3\n19701.7\n19542.7\n19386.3\n19232.5\n19081.2\n18932.2\n18785.5\n18641\n18498.7\n18358.5\n18220.4\n18084.2\n17950\n17817.6\n17687\n17558.2\n17431.1\n17305.7\n17181.9\n17059.7\n16939\n16819.9\n16702.3\n16586.1\n16471.2\n16357.8\n16245.7\n16134.9\n16025.4\n15917.1\n15810.1\n15704.3\n15599.6\n15496.1\n15393.7\n15292.4\n15192.2\n15093.1\n14995\n14897.9\n14801.8\n14706.7\n14612.5\n14519.3\n14427\n14335.6\n14245.1\n14155.5\n14066.7\n13978.8\n13891.7\n13805.4\n13719.9\n13635.2\n13551.3\n13468.2\n13385.8\n13304.1\n13223.2\n13142.9\n13063.4\n12984.6\n12906.4\n12829\n12752.1\n12676\n12600.4\n12525.5\n12451.3\n12

In [84]:
import os

f = open("example.txt", 'r')
i = 0

content = f.readlines()
for line in content:
  if i % 8 == 0:
    print('\n')
  line = line.split(' ')
  for el in line:
    print(el.rjust(12), end='')
  i += 1




           0     1.13272     1.28305     1.45333     1.64621     1.86469     2.11216           0           
     1.13272           0           0           0           0           0           0     3.06966           
     1.28305           0           0           0           0           0           0     3.47706           
     1.45333           0           0           0           0           0           0     3.93852           
     1.64621           0           0           0           0           0           0     4.46122           
     1.86469           0           0           0           0           0           0      5.0533           
     2.11216           0           0           0           0           0           0     5.72396           
           0     3.06966     3.47706     3.93852     4.46122      5.0533     5.72396           0           


           0     1.13272     1.28305     1.45333     1.64621     1.86469     2.11216           0           
     1.13272    0.556334